# TMDB Movie Data Analysis

## Project Overview
This project challenges us to build a movie data analysis pipeline using Python and Pandas. We will fetch movie-related data from the TMDB API, clean and transform the dataset, and implement key performance indicators (KPIs) to identify the best and worst movies based on financial and popularity metrics.

## Objectives
1. **API Data Extraction**: Fetch movie data from the TMDB API.
2. **Data Cleaning & Transformation**: Process and structure the data for analysis.
3. **Exploratory Data Analysis (EDA)**: Perform an initial exploration to understand trends.
4. **Advanced Filtering & Ranking**: Identify the best and worst movies based on financial and popularity metrics.
5. **Franchise & Director Analysis**: Assess how franchises and directors perform over time.
6. **Visualization & Insights**: Present key findings using visualizations.


## Setup and Imports
First, we import the necessary libraries and set up our environment.


In [1]:
import os
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv


load_dotenv()

API_KEY = os.getenv('TMDB_API_KEY')
BASE_URL = "https://api.themoviedb.org/3"

if not API_KEY:
    print("WARNING: TMDB_API_KEY not found in environment variables.")
else:
    print("API Key loaded successfully.")


API Key loaded successfully.


## Step 1: Fetch Movie Data from API
We need to fetch data for a specific list of movies provided in the assignment. We will define functions to fetch details for each movie ID, including credits (cast and crew).


In [2]:
def fetch_movie_details(movie_id):
    """
    Fetches details for a specific movie ID, including credits.
    """
    if not API_KEY:
        raise ValueError("TMDB_API_KEY not found.")
        
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language=en-US&append_to_response=credits"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching movie {movie_id}: {e}")
        return None


In [3]:
def fetch_specific_movies(movie_ids):
    """
    Fetches data for a list of movie IDs.
    """
    movies = []
    for i, movie_id in enumerate(movie_ids):
        data = fetch_movie_details(movie_id)
        if data:
            movies.append(data)
    return movies


In [4]:
# List of movie ids to fetch
movie_ids = [0, 299534, 19995, 140607, 299536, 597, 135397, 420818, 24428, 168259, 99861, 284054, 12445, 181808, 330457, 351286, 109445, 321612, 260513]

raw_movies_data = fetch_specific_movies(movie_ids)

Error fetching movie 0: 404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/0?api_key=ec1db5cde209ace2122b5f7bd9c3f18c&language=en-US&append_to_response=credits


In [5]:
# storing data in a DataFrame
df = pd.DataFrame(raw_movies_data)

## Step 2: Data Cleaning and Preprocessing
Now that we have the raw data, we need to clean it. This involves:
1.  Dropping irrelevant columns.
2.  Extracting data from JSON-like columns (genres, production companies, etc.).
3.  **Inspecting extracted columns** for anomalies.
4.  Handling missing or incorrect data (e.g., 0 budget).
5.  **Handling vote_count = 0**.
6.  Converting data types.
7.  Calculating new metrics like ROI and Profit.


In [6]:
# Helper function to extract names from JSON-like columns
def extract_names(x):
    """Extracts names from a list of dictionaries."""
    if isinstance(x, list):
        return "|".join([i['name'] for i in x if 'name' in i])
    return ""


In [7]:
# Create DataFrame and drop irrelevant columns
df = pd.DataFrame(raw_movies_data)

drop_cols = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

print(f"Initial DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")


Initial DataFrame shape: (18, 22)
Columns: ['backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'id', 'origin_country', 'original_language', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count', 'credits']


In [8]:
# Extract and clean JSON-like columns
json_cols = ['genres', 'belongs_to_collection', 'production_countries', 'production_companies', 'spoken_languages']

for col in json_cols:
    if col in df.columns:
        # special case for belongs_to_collection which is a dict, not list
        if col == 'belongs_to_collection':
            df[col] = df[col].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else "")
        else:
            # other columns which are dict
            df[col] = df[col].apply(extract_names)

In [9]:
# Inspect extracted columns using value_counts() to identify anomalies
for col in json_cols:
    if col in df.columns:
        print(df[col].value_counts().head(5))

genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Science Fiction             2
Action|Adventure|Fantasy|Science Fiction     1
Drama|Romance                                1
Name: count, dtype: int64
belongs_to_collection
The Avengers Collection     4
Star Wars Collection        2
                            2
Frozen Collection           2
Jurassic Park Collection    2
Name: count, dtype: int64
production_countries
United States of America                   16
United States of America|United Kingdom     1
United Kingdom|United States of America     1
Name: count, dtype: int64
production_companies
Marvel Studios                                                                          5
Walt Disney Animation Studios                                                           2
Dune Entertainment|Lightstorm Entertainment|20th Century Fox|Ingenious Film Partners    1
Paramount Pictures|20th Century Fox|Lightstorm Entertainme

In [10]:
# Convert column datatypes
numeric_cols = ['budget', 'id', 'popularity', 'revenue', 'vote_average', 'vote_count', 'runtime']
for col in numeric_cols:
    df[col] = pd.to_numeric(df.get(col, 0), errors='coerce')

In [11]:
# Convert release_date to datetime and extract year
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [12]:
# checking the datatypes after conversion
print(df[numeric_cols + ['release_date', 'release_year']].dtypes)

budget                   int64
id                       int64
popularity             float64
revenue                  int64
vote_average           float64
vote_count               int64
runtime                  int64
release_date    datetime64[ns]
release_year             int32
dtype: object


In [13]:
# Handling unrealistic values and create derived columns
# Replace 0 with NaN for budget, revenue, runtime
for col in ['budget', 'revenue', 'runtime']:
    df[col] = df[col].replace(0, pd.NA)

# Convert budget and revenue to million USD
df['budget_musd'] = df['budget'] / 1000000
df['revenue_musd'] = df['revenue'] / 1000000

# Handle vote_count = 0 (set vote_average to 0 if no votes)
if 'vote_count' in df.columns and 'vote_average' in df.columns:
    df.loc[df['vote_count'] == 0, 'vote_average'] = 0

# Replace placeholders in overview/tagline
for col in ['overview', 'tagline']:
    if col in df.columns:
        df[col] = df[col].replace(['No Data', ''], pd.NA)

In [14]:
# Budget and Revenue statistics (in MUSD)
print(df[['budget_musd', 'revenue_musd']].describe())

       budget_musd  revenue_musd
count    18.000000     18.000000
mean    213.777778   1691.831828
std      61.959717    521.062285
min     125.000000   1243.225667
25%     162.500000   1334.901927
50%     200.000000   1484.541738
75%     243.000000   1957.195640
max     356.000000   2923.706026


In [15]:
# Remove duplicates and drop rows with unknown 'id' or 'title'
df = df.drop_duplicates(subset='id')
df = df.dropna(subset=['id', 'title'])

# Keep only rows where at least 10 columns have non-NaN values
df = df.dropna(thresh=10)

# Filtering to include only 'Released' movies
if 'status' in df.columns:
    # a boolean mask for released movies
    is_released = df['status'] == 'Released'
    
    # Apply the mask to filter the DataFrame
    df = df[is_released]
    
    # Drop the status column as it's no longer needed
    df = df.drop(columns=['status'])

In [16]:
# DataFrame shape after filtering: 
df.shape

(18, 24)

In [17]:
# Helper functions for extracting credits information
def get_director(x):
    """Extracts director name from credits."""
    if isinstance(x, dict) and 'crew' in x:
        for crew in x['crew']:
            if crew.get('job') == 'Director':
                return crew.get('name')
    return ""

In [18]:
def get_cast(x):
    """Extracts top 5 cast members from credits."""
    if isinstance(x, dict) and 'cast' in x:
        return "|".join([c['name'] for c in x['cast'][:5]])
    return ""

In [23]:
df['credits'].head(1)

0    {'cast': [{'adult': False, 'gender': 2, 'id': ...
Name: credits, dtype: object

In [19]:
# Extract credits information (director, cast, sizes)
if 'credits' in df.columns:
    df['director'] = df['credits'].apply(get_director)
    df['cast'] = df['credits'].apply(get_cast)
    df['cast_size'] = df['credits'].apply(lambda x: len(x.get('cast', [])) if isinstance(x, dict) else 0)
    df['crew_size'] = df['credits'].apply(lambda x: len(x.get('crew', [])) if isinstance(x, dict) else 0)

In [20]:
# Sample of extracted credits
print(df[['title', 'director', 'cast']].head())

                          title       director  \
0             Avengers: Endgame  Anthony Russo   
1                        Avatar  James Cameron   
2  Star Wars: The Force Awakens    J.J. Abrams   
3        Avengers: Infinity War      Joe Russo   
4                       Titanic  James Cameron   

                                                cast  
0  Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...  
1  Sam Worthington|Zoe Saldaña|Sigourney Weaver|S...  
2  Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...  
3  Robert Downey Jr.|Chris Evans|Chris Hemsworth|...  
4  Leonardo DiCaprio|Kate Winslet|Billy Zane|Kath...  


In [24]:
# Reorder columns and select final columns
target_cols = [
    'id', 'title', 'tagline', 'release_date', 'release_year', 'genres', 'belongs_to_collection', 
    'original_language', 'budget_musd', 'revenue_musd', 'production_companies', 
    'production_countries', 'vote_count', 'vote_average', 'popularity', 'runtime', 
    'overview', 'spoken_languages', 'poster_path', 'cast', 'cast_size', 'director', 'crew_size'
]

# Select only existing columns from target list
final_cols = [c for c in target_cols if c in df.columns]
df_clean = df[final_cols].copy()

# Reset index
df_clean = df_clean.reset_index(drop=True)

In [25]:
# Final columns: 
list(df_clean.columns)

['id',
 'title',
 'tagline',
 'release_date',
 'release_year',
 'genres',
 'belongs_to_collection',
 'original_language',
 'budget_musd',
 'revenue_musd',
 'production_companies',
 'production_countries',
 'vote_count',
 'vote_average',
 'popularity',
 'runtime',
 'overview',
 'spoken_languages',
 'poster_path',
 'cast',
 'cast_size',
 'director',
 'crew_size']

In [29]:
# Calculate ROI and Profit
df_clean['budget_musd'] = df_clean['budget_musd'].fillna(0)
df_clean['revenue_musd'] = df_clean['revenue_musd'].fillna(0)

df_clean['roi'] = df_clean.apply(
    lambda row: row['revenue_musd'] / row['budget_musd'] if row['budget_musd'] > 0 else 0, 
    axis=1
)
df_clean['profit'] = df_clean['revenue_musd'] - df_clean['budget_musd']

df_clean.shape

(18, 25)

In [ ]:
# ROI and Profit calculated successfully

display(df_clean)

# len(df_clean) # the total movies

,id,title,tagline,release_date,release_year,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,...,runtime,overview,spoken_languages,poster_path,cast,cast_size,director,crew_size,roi,profit
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,2019,Adventure|Science Fiction|Action,The Avengers Collection,en,356.0,2799.439100,...,181,After the devastating events of Avengers: Infi...,English|日本語|,/bR8ISy1O9XQxqiy0fQFw2BX72RQ.jpg,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,105,Anthony Russo,607,7.863593,2443.439100
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,2009,Action|Adventure|Fantasy|Science Fiction,Avatar Collection,en,237.0,2923.706026,...,162,"In the 22nd century, a paraplegic Marine is di...",English|Español,/gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg,Sam Worthington|Zoe Saldaña|Sigourney Weaver|S...,65,James Cameron,990,12.336312,2686.706026
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,2015,Adventure|Action|Science Fiction,Star Wars Collection,en,245.0,2068.223624,...,136,Thirty years after defeating the Galactic Empi...,English,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,183,J.J. Abrams,262,8.441729,1823.223624
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,2018,Adventure|Action|Science Fiction,The Avengers Collection,en,300.0,2052.415039,...,149,As the Avengers and their allies have continue...,English|,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,Robert Downey Jr.|Chris Evans|Chris Hemsworth|...,69,Joe Russo,733,6.841383,1752.415039
4,597,Titanic,Nothing on earth could come between them.,1997-11-18,1997,Drama|Romance,,en,200.0,2264.162353,...,194,101-year-old Rose DeWitt Bukater tells the sto...,English|Français|Deutsch|svenska|Italiano|Pусский,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,Leonardo DiCaprio|Kate Winslet|Billy Zane|Kath...,116,James Cameron,262,11.320812,2064.162353
5,135397,Jurassic World,The park is open.,2015-06-06,2015,Action|Adventure|Science Fiction|Thriller,Jurassic Park Collection,en,150.0,1671.537444,...,124,Twenty-two years after the events of Jurassic ...,English,/rhr4y79GpxQF9IsfJItRXVaoGs4.jpg,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,53,Colin Trevorrow,426,11.143583,1521.537444
6,420818,The Lion King,The king has returned.,2019-07-12,2019,Adventure|Drama|Family|Animation,The Lion King (Reboot) Collection,en,260.0,1662.020819,...,118,"Simba idolizes his father, King Mufasa, and ta...",English,/dzBtMocZuJbjLOXvrl4zGYigDzh.jpg,Chiwetel Ejiofor|John Oliver|Donald Glover|Jam...,20,Jon Favreau,48,6.392388,1402.020819
7,24428,The Avengers,Some assembly required.,2012-04-25,2012,Science Fiction|Action|Adventure,The Avengers Collection,en,220.0,1518.815515,...,143,When an unexpected enemy emerges and threatens...,English|हिन्दी|Pусский,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,113,Joss Whedon,641,6.903707,1298.815515
8,168259,Furious 7,Vengeance hits home.,2015-04-01,2015,Action|Crime|Thriller,The Fast and the Furious Collection,en,190.0,1515.400000,...,139,Deckard Shaw seeks revenge against Dominic Tor...,العربية|English|Español|ภาษาไทย,/ktofZ9Htrjiy0P6LEowsDaxd3Ri.jpg,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,49,James Wan,227,7.975789,1325.400000
9,99861,Avengers: Age of Ultron,A new age has come.,2015-04-22,2015,Action|Adventure|Science Fiction,The Avengers Collection,en,235.0,1405.403694,...,141,When Tony Stark tries to jumpstart a dormant p...,English,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,Robert Downey Jr.|Chris Hemsworth|Mark Ruffalo...,74,Joss Whedon,652,5.980441,1170.403694


In [36]:
# Summary of cleaned data

# Missing values per column:")
print(df_clean.isnull().sum())

id                       0
title                    0
tagline                  0
release_date             0
release_year             0
genres                   0
belongs_to_collection    0
original_language        0
budget_musd              0
revenue_musd             0
production_companies     0
production_countries     0
vote_count               0
vote_average             0
popularity               0
runtime                  0
overview                 0
spoken_languages         0
poster_path              0
cast                     0
cast_size                0
director                 0
crew_size                0
roi                      0
profit                   0
dtype: int64


## Step 3: KPI Implementation & Analysis
We will now analyze the data to identify the best and worst performing movies, analyze franchises, and check specific queries.


### Individual Ranking Implementations
Before we create a reusable UDF, let's implement each ranking individually to understand the logic step by step.

In [ ]:
# 1. Highest Revenue Movies
# Sort by revenue in descending order and take top 5
highest_revenue = df_clean.sort_values('revenue_musd', ascending=False).head(5)
display(highest_revenue[['title', 'revenue_musd']])

In [ ]:
# 2. Highest Budget Movies
# Sort by budget in descending order and take top 5
highest_budget = df_clean.sort_values('budget_musd', ascending=False).head(5)
print("--- Highest Budget Movies ---")
display(highest_budget[['title', 'budget_musd']])

In [ ]:
# 3. Highest Profit Movies (Revenue - Budget)
# Sort by profit in descending order and take top 5
highest_profit = df_clean.sort_values('profit', ascending=False).head(5)
print("--- Highest Profit Movies ---")
display(highest_profit[['title', 'profit']])

In [ ]:
# 4. Lowest Profit Movies (Revenue - Budget)
# Sort by profit in ascending order (lowest first) and take top 5
lowest_profit = df_clean.sort_values('profit', ascending=True).head(5)
print("--- Lowest Profit Movies ---")
display(lowest_profit[['title', 'profit']])

In [ ]:
# 5. Highest ROI (only movies with Budget >= 10M)
# First filter for movies with budget >= 10 million, then sort by ROI descending
high_budget_movies = df_clean[df_clean['budget_musd'] >= 10]
highest_roi = high_budget_movies.sort_values('roi', ascending=False).head(5)
print("--- Highest ROI (Budget >= $10M) ---")
display(highest_roi[['title', 'roi']])

In [ ]:
# 6. Lowest ROI (only movies with Budget >= 10M)
# Filter for movies with budget >= 10 million, then sort by ROI ascending (lowest first)
high_budget_movies = df_clean[df_clean['budget_musd'] >= 10]
lowest_roi = high_budget_movies.sort_values('roi', ascending=True).head(5)
print("--- Lowest ROI (Budget >= $10M) ---")
display(lowest_roi[['title', 'roi']])

In [ ]:
# 7. Most Voted Movies
# Sort by vote_count in descending order and take top 5
most_voted = df_clean.sort_values('vote_count', ascending=False).head(5)
print("--- Most Voted Movies ---")
display(most_voted[['title', 'vote_count']])

In [ ]:
# 8. Highest Rated Movies (only movies with >= 10 votes)
# Filter for movies with at least 10 votes, then sort by vote_average descending
movies_with_votes = df_clean[df_clean['vote_count'] >= 10]
highest_rated = movies_with_votes.sort_values('vote_average', ascending=False).head(5)
print("--- Highest Rated Movies (Votes >= 10) ---")
display(highest_rated[['title', 'vote_average']])

In [ ]:
# 9. Lowest Rated Movies (only movies with >= 10 votes)
# Filter for movies with at least 10 votes, then sort by vote_average ascending (lowest first)
movies_with_votes = df_clean[df_clean['vote_count'] >= 10]
lowest_rated = movies_with_votes.sort_values('vote_average', ascending=True).head(5)
print("--- Lowest Rated Movies (Votes >= 10) ---")
display(lowest_rated[['title', 'vote_average']])

In [ ]:
# 10. Most Popular Movies
# Sort by popularity in descending order and take top 5
most_popular = df_clean.sort_values('popularity', ascending=False).head(5)
print("--- Most Popular Movies ---")
display(most_popular[['title', 'popularity']])

### Creating a Reusable UDF (User-Defined Function)
Notice how the individual implementations above follow a similar pattern:
1. Optionally filter the data (e.g., budget >= 10M, votes >= 10)
2. Sort by a metric (ascending or descending)
3. Take the top N results
4. Display specific columns

We can consolidate this repetitive logic into a single reusable function:

In [ ]:
# Helper function for ranking movies
def rank_movies(df, metric, ascending=False, top_n=5, filter_col=None, filter_val=None):
    """Ranks movies by a specified metric with optional filtering."""
    data = df.copy()
    if filter_col:
        data = data[data[filter_col] >= filter_val]
    
    ranked = data.sort_values(metric, ascending=ascending).head(top_n)
    return ranked[['title', metric]]


In [ ]:
# Best/Worst Performing Movies - Revenue & Budget
print("=== Best/Worst Performing Movies ===")

print("\n--- Highest Revenue ---")
display(rank_movies(df_clean, 'revenue_musd'))

print("\n--- Highest Budget ---")
display(rank_movies(df_clean, 'budget_musd'))


In [ ]:
# Best/Worst Performing Movies - Profit
print("--- Highest Profit ---")
display(rank_movies(df_clean, 'profit'))

print("\n--- Lowest Profit ---")
display(rank_movies(df_clean, 'profit', ascending=True))


In [ ]:
# Best/Worst Performing Movies - ROI (Return on Investment)
print("--- Highest ROI (Budget >= 10M) ---")
display(rank_movies(df_clean, 'roi', filter_col='budget_musd', filter_val=10))

print("\n--- Lowest ROI (Budget >= 10M) ---")
display(rank_movies(df_clean, 'roi', ascending=True, filter_col='budget_musd', filter_val=10))


In [ ]:
# Best/Worst Performing Movies - Votes & Ratings
print("--- Most Voted Movies ---")
display(rank_movies(df_clean, 'vote_count'))

print("\n--- Highest Rated (Votes >= 10) ---")
display(rank_movies(df_clean, 'vote_average', filter_col='vote_count', filter_val=10))

print("\n--- Lowest Rated (Votes >= 10) ---")
display(rank_movies(df_clean, 'vote_average', ascending=True, filter_col='vote_count', filter_val=10))


In [ ]:
# Most Popular Movies
print("--- Most Popular Movies ---")
display(rank_movies(df_clean, 'popularity'))


In [ ]:
# Advanced Movie Filtering - Sci-Fi Action movies starring Bruce Willis
print("=== Advanced Movie Filtering ===")

mask_scifi = df_clean['genres'].str.contains('Science Fiction', na=False)
mask_action = df_clean['genres'].str.contains('Action', na=False)
mask_bruce = df_clean['cast'].str.contains('Bruce Willis', na=False)

bruce_movies = df_clean[mask_scifi & mask_action & mask_bruce].sort_values('vote_average', ascending=False)
print("\n--- Sci-Fi Action movies starring Bruce Willis ---")
display(bruce_movies[['title', 'vote_average', 'release_date']])


In [ ]:
# Advanced Movie Filtering - Uma Thurman & Quentin Tarantino
mask_uma = df_clean['cast'].str.contains('Uma Thurman', na=False)
mask_qt = df_clean['director'].str.contains('Quentin Tarantino', na=False)

uma_qt_movies = df_clean[mask_uma & mask_qt].sort_values('runtime')
print("--- Uma Thurman & Quentin Tarantino Movies (by Runtime) ---")
display(uma_qt_movies[['title', 'runtime', 'release_date']])


In [ ]:
# Franchise vs Standalone Analysis
print("=== Franchise vs Standalone Analysis ===")

df_clean['is_franchise'] = df_clean['belongs_to_collection'].apply(lambda x: True if x else False)

franchise_stats = df_clean.groupby('is_franchise').agg({
    'revenue_musd': 'mean',
    'roi': 'median',
    'budget_musd': 'mean',
    'popularity': 'mean',
    'vote_average': 'mean'
}).rename(index={True: 'Franchise', False: 'Standalone'})

print("\n--- Franchise vs Standalone Stats ---")
display(franchise_stats)


In [ ]:
# Most Successful Franchises
print("=== Most Successful Franchises ===")

franchise_df = df_clean[df_clean['is_franchise']].groupby('belongs_to_collection').agg({
    'title': 'count',
    'budget_musd': ['sum', 'mean'],
    'revenue_musd': ['sum', 'mean'],
    'vote_average': 'mean'
})
franchise_df.columns = ['movie_count', 'total_budget', 'mean_budget', 'total_revenue', 'mean_revenue', 'mean_rating']

print("\n--- Top 5 Franchises by Total Revenue ---")
display(franchise_df.sort_values('total_revenue', ascending=False).head(5))


In [ ]:
# Most Successful Directors
print("=== Most Successful Directors ===")

director_df = df_clean.groupby('director').agg({
    'title': 'count',
    'revenue_musd': 'sum',
    'vote_average': 'mean'
})
director_df.columns = ['movie_count', 'total_revenue', 'mean_rating']

# Filter out empty director if any
if "" in director_df.index:
    director_df = director_df.drop("")

print("\n--- Top 5 Directors by Total Revenue ---")
display(director_df.sort_values('total_revenue', ascending=False).head(5))


## Step 4: Data Visualization
Finally, we visualize the data to better understand the relationships between different variables.


In [ ]:
# Set visualization theme
sns.set_theme(style="whitegrid")


In [ ]:
# Plot 1: Revenue vs Budget
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='budget_musd', y='revenue_musd', hue='is_franchise', alpha=0.7)
plt.title('Revenue vs Budget')
plt.xlabel('Budget (MUSD)')
plt.ylabel('Revenue (MUSD)')
plt.show()


In [ ]:
# Plot 2: ROI Distribution by Genre (Top 5 genres)
# Explode genres first
df_genres = df_clean.assign(genre=df_clean['genres'].str.split('|')).explode('genre')
top_genres = df_genres['genre'].value_counts().head(5).index
df_top_genres = df_genres[df_genres['genre'].isin(top_genres)]

plt.figure(figsize=(12, 6))
sns.boxplot(data=df_top_genres, x='genre', y='roi')
plt.title('ROI Distribution by Top 5 Genres')
plt.ylim(-1, 10)  # Limit y-axis to see distribution better
plt.show()


In [ ]:
# Plot 3: Popularity vs Rating
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='vote_average', y='popularity', alpha=0.6)
plt.title('Popularity vs Rating')
plt.xlabel('Vote Average')
plt.ylabel('Popularity')
plt.show()


In [ ]:
# Plot 4: Franchise vs Standalone Comparison (Bar Chart)
franchise_plot = franchise_stats.reset_index()
franchise_melt = franchise_plot.melt(
    id_vars='is_franchise', 
    value_vars=['revenue_musd', 'budget_musd'], 
    var_name='Metric', 
    value_name='Value (MUSD)'
)

plt.figure(figsize=(10, 6))
sns.barplot(data=franchise_melt, x='Metric', y='Value (MUSD)', hue='is_franchise')
plt.title('Franchise vs Standalone: Revenue & Budget')
plt.show()


In [ ]:
# Plot 5: Yearly Trends in Box Office Performance
yearly_stats = df_clean.groupby('release_year')['revenue_musd'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=yearly_stats, x='release_year', y='revenue_musd', marker='o')
plt.title('Yearly Trends in Box Office Revenue')
plt.xlabel('Year')
plt.ylabel('Total Revenue (MUSD)')
plt.show()


## Conclusion
In this analysis, we fetched movie data from the TMDB API, cleaned it, and performed various analyses to understand what makes a movie successful. We looked at financial metrics like Revenue and ROI, as well as popularity and ratings. We also compared franchises vs. standalone movies and identified top directors.
